# ネットワークを遺伝子単位で出力する


In [1]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir("../")

print(os.getcwd())

/mnt/e/TSUMUGI/notebooks/notebools-web
/mnt/e/TSUMUGI


In [2]:
P = print
from pprint import pprint as PP
from collections import Counter as C
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import json
import pickle

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
overlap = pickle.load(open("data/overlapped_ratios_filtered.pkl", "rb"))
df_overlap = pd.DataFrame(
    overlap, columns=["marker1", "marker2", "overlap_ratio", "overlapped_mp_number", "overlapped_mp"]
)
df_overlap  # 240358 rows × 5 columns

,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
0,Uap1,Eif4g2,0.250,3,"[abnormal lens morphology, embryonic lethality..."
1,Uap1,Pdcd7,0.188,3,"[cataract, embryonic lethality prior to organo..."
2,Uap1,Chordc1,0.250,3,"[cataract, embryonic lethality prior to organo..."
3,Uap1,Epc2,0.176,3,"[abnormal lens morphology, cataract, preweanin..."
4,Uap1,Alas1,0.333,3,"[cataract, embryonic lethality prior to organo..."
...,...,...,...,...,...
240353,Tars2,Dmap1,1.000,3,"[embryonic lethality prior to organogenesis, e..."
240354,Tars2,Exosc8,0.750,3,"[embryonic lethality prior to organogenesis, e..."
240355,Arf3,Cdnf,0.600,3,"[abnormal retina morphology, abnormal retina v..."
240356,Zfp407,Mepce,0.750,3,"[embryonic lethality prior to organogenesis, i..."


In [6]:
marker_mp = pickle.load(open("data/marker_mp.pkl", "rb"))
marker_mp = pd.DataFrame(marker_mp.items(), columns=["marker_symbol", "mp_term"])
marker_mp  # 7746 rows × 2 columns

,marker_symbol,mp_term
0,Uap1,"[abnormal lens morphology, cataract, embryonic..."
1,6430548M08Rik,"[abnormal kidney morphology, abnormal lymph no..."
2,Stac,[abnormal locomotor behavior]
3,Slc24a4,"[decreased prepulse inhibition, decreased red ..."
4,Uhrf2,"[decreased body length, decreased bone mineral..."
...,...,...
7741,Tm9sf1,[increased total body fat amount]
7742,Eci3,[increased circulating alanine transaminase le...
7743,Stambp,[increased startle reflex]
7744,Wdr45b,[abnormal embryo size]


In [7]:
# DFをdictに変換
marker_mp_dict = dict(zip(marker_mp.marker_symbol, marker_mp.mp_term))

## ひとつの遺伝子をハンドルする


In [8]:
gene_symbol = "Rab10"

In [9]:
P(sum(df_overlap.marker1 == gene_symbol))
P(sum(df_overlap.marker2 == gene_symbol))

7
67


In [10]:
import networkx as nx

In [11]:
G = nx.from_pandas_edgelist(df_overlap, "marker1", "marker2")

In [12]:
# ノードAと直接つながっているノードのみを取得
neighbors = list(G.neighbors(gene_symbol))
subgraph_nodes = [gene_symbol] + neighbors
subgraph = G.subgraph(subgraph_nodes)

P(subgraph)

Graph with 75 nodes and 845 edges


In [13]:
P(subgraph.nodes())

['Cdca5', 'Nufip2', 'U2af1', 'Pacsin2', 'Haus4', 'Dhx15', 'Hars2', 'U2af2', 'Ddx46', 'Dhfr', 'Gtpbp4', 'Ints3', 'Trappc11', 'Rab10', 'Cdc37', 'Thoc3', 'Atl2', 'Tomm40', 'Lmbrd1', 'Ccdc59', 'Atp6v1b2', 'Syncrip', 'Btf3', 'Mipep', 'Rnaseh2a', 'Osbp', 'Cox20', 'Nlk', 'Clptm1', 'Wdr38', 'Eid2b', 'Otoa', 'Ppie', 'Sepsecs', 'Mccc2', 'Wee1', 'Gins4', 'Fbxo11', 'Chp1', 'Rpain', 'Snap23', 'Fbxo5', 'Hmbs', 'Med10', 'Med25', 'Ebna1bp2', 'Zc3h7a', 'Asxl1', 'Tomm22', 'Smarce1', 'Cct3', 'Atp8b1', 'Rnf10', 'Ints8', 'Mat2a', 'Scfd1', 'Wars1', 'Plekhf2', 'Tfrc', '0610010K14Rik', 'Med28', 'Srprb', 'Ap4e1', 'Rrbp1', 'H2az1', 'Prpf39', 'Prpf40a', 'Mtmr3', 'Ercc3', 'Sptb', 'Atp2b1', 'Zmynd8', 'Rac1', 'Dnm2', 'Mcmbp']


In [14]:
# nodesを用意

node_json = []
for node in subgraph.nodes():
    annotation = marker_mp_dict[node]
    node_color = 1 if node == gene_symbol else 0
    node_json.append({"data": {"id": node, "label": node, "node_color": node_color, "annotation": annotation}})

P(node_json[0])

{'data': {'id': 'Cdca5', 'label': 'Cdca5', 'node_color': 0, 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'embryonic lethality prior to organogenesis', 'embryonic lethality prior to tooth bud stage', 'increased circulating alanine transaminase level', 'increased circulating chloride level', 'increased circulating sodium level', 'preweaning lethality, complete penetrance']}}


In [15]:
# edgesを用意
df_overlap["id"] = df_overlap.apply(lambda row: "-".join(sorted([row["marker1"], row["marker2"]])), axis=1)

edge_info = pd.DataFrame({"id": ["-".join(sorted([a, b])) for a, b in subgraph.edges()]}).merge(df_overlap, on="id")
edge_json = []
for edge in edge_info.itertuples():
    edge_json.append(
        {
            "data": {
                "source": edge.marker1,
                "target": edge.marker2,
                "edge_size": edge.overlap_ratio,
                "annotation": edge.overlapped_mp,
            }
        }
    )
P(edge_json[:3])

[{'data': {'source': 'Lmbrd1', 'target': 'Cdca5', 'edge_size': 0.3, 'annotation': ['embryonic lethality prior to organogenesis', 'embryonic lethality prior to tooth bud stage', 'preweaning lethality, complete penetrance']}}, {'data': {'source': 'Rnf10', 'target': 'Cdca5', 'edge_size': 0.044, 'annotation': ['abnormal auditory brainstem response', 'increased circulating alanine transaminase level', 'increased circulating sodium level']}}, {'data': {'source': 'U2af2', 'target': 'Cdca5', 'edge_size': 0.25, 'annotation': ['abnormal auditory brainstem response', 'embryonic lethality prior to organogenesis', 'preweaning lethality, complete penetrance']}}]


In [16]:
# Cytoscape.js用のJSON形式に変換
cytoscape_data = node_json + edge_json

In [17]:
# JSONに変換
cytoscape_json = json.dumps(cytoscape_data, indent=2)
print(cytoscape_json)

[
  {
    "data": {
      "id": "Cdca5",
      "label": "Cdca5",
      "node_color": 0,
      "annotation": [
        "abnormal auditory brainstem response",
        "abnormal lens morphology",
        "embryonic lethality prior to organogenesis",
        "embryonic lethality prior to tooth bud stage",
        "increased circulating alanine transaminase level",
        "increased circulating chloride level",
        "increased circulating sodium level",
        "preweaning lethality, complete penetrance"
      ]
    }
  },
  {
    "data": {
      "id": "Nufip2",
      "label": "Nufip2",
      "node_color": 0,
      "annotation": [
        "abnormal embryo size",
        "abnormal neural tube closure",
        "decreased circulating alkaline phosphatase level",
        "decreased mean corpuscular hemoglobin",
        "decreased prepulse inhibition",
        "increased startle reflex",
        "preweaning lethality, complete penetrance"
      ]
    }
  },
  {
    "data": {
      "id": "U

In [18]:
# Path("Rab10.json").write_text(cytoscape_json)

## 一括処理


In [19]:
import shutil

output_dir = Path("data/network/gene_symbol")
# remove network directory
if output_dir.exists():
    shutil.rmtree(output_dir)

output_dir.mkdir(exist_ok=True, parents=True)


In [20]:
gene_symbols = df_overlap.marker1.unique().tolist()
P(gene_symbols)
gene_symbols += df_overlap.marker2.unique().tolist()
gene_symbols = list(set(gene_symbols))
gene_symbols.sort()  # 以下のfor文で、どこまで遺伝子が処理されたのか途中経過を見積もるためのソート
P(gene_symbols[:3])
P(len(gene_symbols))

['Uap1', '6430548M08Rik', 'Slc24a4', 'Uhrf2', 'Nhlrc3', 'Tonsl', 'C1qtnf1', 'Col18a1', 'Prdm8', 'Chsy3', 'Kcne1', 'Agtr1a', 'Pramel13', 'Ddx6', 'Krt1', 'Spryd4', 'Timm13', 'Usp32', 'Angptl3', 'Zwint', 'Usp26', 'Eif4g2', 'Fa2h', 'Nrxn1', 'Ptpru', 'Pdcd7', 'Ncbp3', 'Tut4', 'Abtb1', 'Tle4', 'Csmd3', 'Slc25a1', 'Krt27', 'Myo6', 'Plekhf2', 'Polr1d', 'Ndrg4', 'Slc38a7', 'Ghrhr', 'Awat2', 'Ap4e1', 'Brap', 'Mink1', 'Mrgprb2', 'Kcnma1', 'Galnt18', 'Wdr41', 'Mogs', 'Bicc1', 'Adpgk', 'Phrf1', 'Fkbp11', 'Rpe', 'Aadac', 'Spock1', 'Ate1', 'Odad2', 'Fgf3', 'Irf8', 'Safb2', 'Tmem248', 'Col4a5', 'Itgae', 'Susd6', 'Esyt3', 'Nlgn3', 'Prmt7', 'Snx15', 'Kiss1r', 'Tcf7l1', 'Atp2a2', '2610008E11Rik', 'Prmt6', 'Baz2a', 'Ubiad1', 'Or1e1c', 'Cog6', 'Ctsa', 'Zfat', 'Cars2', 'Dbn1', 'Fbxw18', 'Sfmbt2', 'Micos13', 'Cyp19a1', 'Gapvd1', 'Igsf9b', 'Slc29a3', 'Mboat2', 'Rapgef2', 'Appbp2', 'Trh', 'Gxylt1', 'Cfb', 'Slc38a10', 'Tmprss6', 'Sox14', 'Dmd', 'Smdt1', 'Dhx29', 'Atp6v1c1', 'Strbp', 'Itgb6', 'Cyb561', 'Elovl5',

In [21]:
# 前処理
G = nx.from_pandas_edgelist(df_overlap, "marker1", "marker2")
df_overlap["id"] = df_overlap.apply(lambda row: "-".join(sorted([row["marker1"], row["marker2"]])), axis=1)

In [22]:
for gene_symbol in gene_symbols:
    # ノードAと直接つながっているノードのみを取得
    neighbors = list(G.neighbors(gene_symbol))
    subgraph_nodes = [gene_symbol] + neighbors
    subgraph = G.subgraph(subgraph_nodes)

    # nodesを用意
    node_json = []
    for node in subgraph.nodes():
        annotation = marker_mp_dict[node]
        node_color = 1 if node == gene_symbol else 0
        node_json.append({"data": {"id": node, "label": node, "node_color": node_color, "annotation": annotation}})

    # edgesを用意
    edge_info = pd.DataFrame({"id": ["-".join(sorted([a, b])) for a, b in subgraph.edges()]}).merge(df_overlap, on="id")
    edge_json = []
    for edge in edge_info.itertuples():
        edge_json.append(
            {
                "data": {
                    "source": edge.marker1,
                    "target": edge.marker2,
                    "edge_size": edge.overlap_ratio,
                    "annotation": edge.overlapped_mp,
                }
            }
        )

    ### EdgeとNodeを統合して、出力
    # Combine node and edge

    network_json = node_json + edge_json
    # Output as JSON fil
    output_dir_mp_term = output_dir / f"{gene_symbol}.json"
    if network_json:
        with open(output_dir_mp_term, "w") as f:
            json.dump(network_json, f, indent=2)

# 6m 52s

In [ ]:
%%bash

ls -lh data/network/gene_symbol/ | head # total 8.0G
head -n 20 data/network/gene_symbol/Rab10.json

total 8.0G
-rwxrwxrwx 1 aki aki  3.5M Dec  4 12:46 0610010K14Rik.json
-rwxrwxrwx 1 aki aki  2.7M Dec  4 12:46 0610040J01Rik.json
-rwxrwxrwx 1 aki aki   30K Dec  4 12:46 1500009L16Rik.json
-rwxrwxrwx 1 aki aki  298K Dec  4 12:46 1700003F12Rik.json
-rwxrwxrwx 1 aki aki  103K Dec  4 12:46 1700008P02Rik.json
-rwxrwxrwx 1 aki aki  991K Dec  4 12:46 1700010I14Rik.json
-rwxrwxrwx 1 aki aki  5.3M Dec  4 12:46 1700025G04Rik.json
-rwxrwxrwx 1 aki aki   16M Dec  4 12:46 1700034E13Rik.json
-rwxrwxrwx 1 aki aki   52K Dec  4 12:46 1700069L16Rik.json
[
  {
    "data": {
      "id": "Cdca5",
      "label": "Cdca5",
      "node_color": 0,
      "annotation": [
        "abnormal auditory brainstem response",
        "abnormal lens morphology",
        "embryonic lethality prior to organogenesis",
        "embryonic lethality prior to tooth bud stage",
        "increased circulating alanine transaminase level",
        "increased circulating chloride level",
        "increased circulating sodium level",


In [24]:
Path("data/available_gene_symbols.txt").write_text("\n".join(gene_symbols) + "\n")

29094

# Debug


In [ ]:
# path_overlap = Path("data", "overlap_ratios_filtered.csv")
# df_overlap = pd.read_csv(path_overlap)

overlap = pickle.load(open("data/overlapped_ratios_filtered.pkl", "rb"))
df_overlap = pd.DataFrame(
    overlap, columns=["marker1", "marker2", "overlap_ratio", "overlapped_mp_number", "overlapped_mp"]
)
P(len(df_overlap))  # 184250
df_overlap.head(3)

184250


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
0,Rhd,Ap4e1,0.071,5,"[decreased circulating free fatty acids level,..."
1,Rhd,Asxl1,0.136,3,"[decreased hemoglobin content, decreased mean ..."
2,Rhd,Abcg8,0.091,4,"[decreased hemoglobin content, decreased mean ..."


In [ ]:
path_mp_terms = list(Path("data", "mp_term_name").glob("*.csv"))
print(path_mp_terms[:3])
print(len(path_mp_terms))  # 670

[PosixPath('data/mp_term_name/abnormal_abdominal_wall_morphology.csv'), PosixPath('data/mp_term_name/abnormal_adipose_tissue_amount.csv'), PosixPath('data/mp_term_name/abnormal_adrenal_gland_morphology.csv')]
670


In [ ]:
# path_mp_term = Path("data", "mp_term_name", "decreased_circulating_glucose_level.csv")
path_mp_term = Path("data", "mp_term_name", "increased_circulating_glucose_level.csv")
mp_term = path_mp_term.stem
print(mp_term)


increased_circulating_glucose_level


In [ ]:
df_marker_effect = pd.read_csv(path_mp_term)
df_marker_effect = df_marker_effect[["marker_symbol", "effect_size"]].drop_duplicates()

# effect_sizeがNanの行を削除
df_marker_effect = df_marker_effect.dropna(subset=["effect_size"])

print(len(df_marker_effect))

df_marker_effect.head(3)


73


,marker_symbol,effect_size
0,Zfp513,0.200596
1,Dbnl,0.828539
2,Oas1g,0.922229


In [ ]:
# print(df_mp["effect_size"].describe())

In [ ]:
# Absolute value of effect size

df_marker_effect.loc[:, "effect_size"] = df_marker_effect["effect_size"].abs()

In [ ]:
# print(df_overlap[df_overlap["intersect_count"] > 5 & df_overlap["overlap_ratio"] > 0.5])

In [ ]:
# df_overlap_filterd = df_overlap[df_overlap["overlapped_mp_number"] > 0]
# df_overlap_filterd = df_overlap[(df_overlap["overlap_ratio"] > 0.25)]
df_overlap_filterd = df_overlap

In [ ]:
df_filtered = df_overlap_filterd[
    (df_overlap_filterd["marker1"].isin(df_marker_effect["marker_symbol"]))
    & (df_overlap_filterd["marker2"].isin(df_marker_effect["marker_symbol"]))
]

In [ ]:
print(len(df_filtered))

df_filtered.head(3)

213


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
2704,Zfyve19,Rnf10,0.079,5,"[abnormal behavior, decreased exploration in n..."
2712,Zfyve19,Usp31,0.136,3,"[decreased exploration in new environment, dec..."
2722,Zfyve19,Ky,0.273,3,"[abnormal behavior, decreased exploration in n..."


## 出力


### Node を作成する


In [ ]:
# merge mp term
df_marker1 = df_filtered[["marker1"]]
df_marker2 = df_filtered[["marker2"]]
df_node_marker1 = pd.merge(df_marker1, df_marker_effect, left_on="marker1", right_on="marker_symbol", how="inner")[
    ["marker_symbol"]
]
df_node_marker2 = pd.merge(df_marker2, df_marker_effect, left_on="marker2", right_on="marker_symbol", how="inner")[
    ["marker_symbol"]
]

df_node = pd.concat([df_node_marker1, df_node_marker2], axis=0).drop_duplicates()

import pickle

marker_mp = pickle.load(open("data/marker_mp.pkl", "rb"))
marker_mp = pd.DataFrame(marker_mp.items(), columns=["marker_symbol", "mp_term"])

df_node = pd.merge(df_node, marker_mp, how="inner", on="marker_symbol")
df_node = pd.merge(df_node, df_marker_effect, how="inner", on="marker_symbol")

df_node[:3]


,marker_symbol,mp_term,effect_size
0,Zfyve19,"[abnormal behavior, decreased exploration in n...",0.695784
1,Csnk1g2,"[abnormal auditory brainstem response, abnorma...",0.665518
2,Bbs5,"[abnormal bone structure, abnormal gait, abnor...",1.902677


In [ ]:
# JSON形式に変換
node_json = []
for _, row in df_node.iterrows():
    node_json.append(
        {
            "data": {
                "id": row["marker_symbol"],
                "label": row["marker_symbol"],
                "annotation": row["mp_term"],
                "node_color": row["effect_size"],
            }
        }
    )

print(node_json[:3])

[{'data': {'id': 'Zfyve19', 'label': 'Zfyve19', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'decreased locomotor activity', 'decreased thigmotaxis', 'increased circulating glucose level'], 'node_color': 0.695783774710263}}, {'data': {'id': 'Csnk1g2', 'label': 'Csnk1g2', 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'abnormal retina blood vessel morphology', 'abnormal retina morphology', 'abnormal retina vasculature morphology', 'decreased total retina thickness', 'increased cardiac muscle contractility', 'increased circulating aspartate transaminase level', 'increased circulating glucose level', 'increased circulating phosphate level', 'increased circulating potassium level', 'increased heart rate variability', 'increased mean corpuscular volume', 'increased pulmonary respiratory rate', 'persistence of hyaloid vascular system'], 'node_color': 0.665518283420352}}, {'data': {'id': 'Bbs5', 'label': 'Bbs5', 'annotatio

### Edge を作成する


In [ ]:
df_edge = df_filtered[["marker1", "marker2", "overlap_ratio", "overlapped_mp"]]
print(len(df_edge))
df_edge.head(3)

213


,marker1,marker2,overlap_ratio,overlapped_mp
2704,Zfyve19,Rnf10,0.079,"[abnormal behavior, decreased exploration in n..."
2712,Zfyve19,Usp31,0.136,"[decreased exploration in new environment, dec..."
2722,Zfyve19,Ky,0.273,"[abnormal behavior, decreased exploration in n..."


In [ ]:
# JSON形式に変換
edge_json = []
for _, row in df_edge.iterrows():
    edge_json.append(
        {
            "data": {
                "source": row["marker1"],
                "target": row["marker2"],
                "annotation": row["overlapped_mp"],
                "edge_size": row["overlap_ratio"],
            }
        }
    )

print(edge_json[:3])

[{'data': {'source': 'Zfyve19', 'target': 'Rnf10', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'decreased locomotor activity', 'decreased thigmotaxis', 'increased circulating glucose level'], 'edge_size': 0.079}}, {'data': {'source': 'Zfyve19', 'target': 'Usp31', 'annotation': ['decreased exploration in new environment', 'decreased locomotor activity', 'increased circulating glucose level'], 'edge_size': 0.136}}, {'data': {'source': 'Zfyve19', 'target': 'Ky', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'increased circulating glucose level'], 'edge_size': 0.273}}]


### Edge と Node を統合して、出力


In [ ]:
# Combine node and edge

network_json = node_json + edge_json

In [ ]:
print(len(network_json))
print(network_json[:3])
print(network_json[-3:])

269
[{'data': {'id': 'Zfyve19', 'label': 'Zfyve19', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'decreased locomotor activity', 'decreased thigmotaxis', 'increased circulating glucose level'], 'node_color': 0.695783774710263}}, {'data': {'id': 'Csnk1g2', 'label': 'Csnk1g2', 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'abnormal retina blood vessel morphology', 'abnormal retina morphology', 'abnormal retina vasculature morphology', 'decreased total retina thickness', 'increased cardiac muscle contractility', 'increased circulating aspartate transaminase level', 'increased circulating glucose level', 'increased circulating phosphate level', 'increased circulating potassium level', 'increased heart rate variability', 'increased mean corpuscular volume', 'increased pulmonary respiratory rate', 'persistence of hyaloid vascular system'], 'node_color': 0.665518283420352}}, {'data': {'id': 'Bbs5', 'label': 'Bbs5', 'annot

In [ ]:
# Output as JSON file
Path("data/network").mkdir(exist_ok=True, parents=True)
with open(f"data/network/{mp_term}.json", "w") as f:
    json.dump(network_json, f, indent=2)

In [32]:
marker_mp

,marker_symbol,mp_term
0,Dpf2,"[Abnormal craniofacial morphology, Abnormal em..."
1,Gna13,"[Abnormal embryo size, Abnormal visceral yolk ..."
2,Dynlrb2,"[Increased fasting circulating glucose level, ..."
3,Slc16a2,"[Decreased circulating cholesterol level, Decr..."
4,Galr1,"[Decreased lymphocyte cell number, Enlarged sp..."
...,...,...
7167,Adgrb1,[Increased circulating insulin level]
7168,Amigo3,[Enlarged heart]
7169,Hap1,"[Preweaning lethality, complete penetrance]"
7170,Slc23a3,[Abnormal lung morphology]


In [38]:
for data in node_json:
    if "Lepr" == data["data"]["id"]:
        print(data)

{'data': {'id': 'Lepr', 'label': 'Lepr', 'annotation': ['Abnormal bone structure', 'Abnormal gait', 'Abnormal locomotor behavior', 'Decreased bone mineral content', 'Decreased bone mineral density', 'Decreased circulating chloride level', 'Decreased exploration in new environment', 'Decreased heart weight', 'Decreased lean body mass', 'Decreased leukocyte cell number', 'Decreased locomotor activity', 'Decreased lymphocyte cell number', 'Decreased spleen weight', 'Enlarged liver', 'Female infertility', 'Impaired righting response', 'Increased anxiety-related response', 'Increased circulating alanine transaminase level', 'Increased circulating alkaline phosphatase level', 'Increased circulating aspartate transaminase level', 'Increased circulating calcium level', 'Increased circulating cholesterol level', 'Increased circulating fructosamine level', 'Increased circulating glucose level', 'Increased circulating hdl cholesterol level', 'Increased circulating iron level', 'Increased circulat

In [163]:
for data in edge_json:
    if "Herc1" == data["data"]["source"]:
        print(data)
    if "Herc1" == data["data"]["target"]:
        print(data)

{'data': {'source': 'Camsap3', 'target': 'Herc1', 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'cataract', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.31}}
{'data': {'source': 'Prdm14', 'target': 'Herc1', 'annotation': ['decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'impaired glucose tolerance', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.273}}
{'data': {'source': 'Kash5', 'target': 'Herc1', 'annotation': ['decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.286}}
{'data': {'source': 'Trim37', 'target': 'Herc1', 'annotation': ['abnormal auditory brainstem response', 'abnor

In [150]:
for data in edge_json:
    if "Stoml2" == data["data"]["source"]:
        print(data)
    if "Stoml2" == data["data"]["target"]:
        print(data)

{'data': {'source': 'Vgf', 'target': 'Stoml2', 'annotation': ['decreased prepulse inhibition', 'increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.25}}
{'data': {'source': 'Fbxo38', 'target': 'Stoml2', 'annotation': ['increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.214}}
{'data': {'source': 'Stoml2', 'target': 'Clcn1', 'annotation': ['decreased prepulse inhibition', 'increased circulating aspartate transaminase level', 'increased circulating bilirubin level', 'male infertility'], 'edge_size': 0.182}}
{'data': {'source': 'Stoml2', 'target': 'Lepr', 'annotation': ['increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.073}}
{'data': {'source': 'Stoml2', 'target': 'Atp13a3', 'annotation': ['increased circulating alanine tra